## 로우 데이터

In [ ]:
# 중식계 컬럼 정리
년 category
월 category
주 category
요일 category
공휴일전후 binary
frequency_sin_year	numeric	< np.sin((2 * np.pi) * 현재날짜_to초/하루_to초) > 
frequency_cos_year	numeric	< np.cos((2 * np.pi) * 현재날짜_to초/하루_to초) > 

# 년월일주, 계절
# 1. 일자 관련 컬럼
train_x['년'] = train_x['일자'].dt.year
train_x['월'] = train_x['일자'].dt.month
train_x['일'] = train_x['일자'].dt.day
train_x['주'] = train_x['일자'].dt.isocalendar().week
train_x['요일'] = train_x['일자'].dt.weekday
# 계절
season = []
for i in train_x['월']:
    if i in [3,4,5]:
        season.append(0)
    elif i in [6,7,8]:
        season.append(1)
    elif i in [9,10,11]:
        season.append(2)
    else:
        season.append(3)
train_x['계절'] = season
weekofmonth = []
for i in train_x["주"]:
    weekofmonth.append((i-1) % 4)

train_x["before_2020"] = [1 if i >= 2020 else 0 for i in train_x['년']]

# ===== 공휴일전후여부 컬럼추가 (공휴일전후:1 / 공휴일전후아님:0) =====
a = []
for n in range(len(df_train)):
    if n==0:   #첫행은 앞의 행이 없으므로, ‘뒷행-1’값이랑만 일치하면 평일
        if (df_train.iloc[n,1]) == (df_train.iloc[n+1,1]-1):
            a.append(0)
        else:
            a.append(1)
        
    elif n==len(df_train)-1:   #마지막행은 뒷 행 없음, ‘앞행+1’ 값만 일치하면 평일
        if (df_train.iloc[n,1] == (df_train.iloc[n-1,1]+1)):
            a.append(0)
        else:
            a.append(1)
            
    elif df_train.iloc[n,1] == 0:   #월요일(0)의 경우, ‘뒷행-1’값은 0, ‘앞행+1’값은 5여야 평일
        if (df_train.iloc[n+1,1]-1 == 0) and (df_train.iloc[n-1,1]+1 == 5):
            a.append(0)
        else:
            a.append(1)
            
    elif df_train.iloc[n,1] == 4:   #금요일(4)의 경우, ‘앞행+1’값은 4, ‘뒷행-1’ 값은 -1이어야 평일
        if (df_train.iloc[n+1,1] -1 == -1) and (df_train.iloc[n-1,1]+1 == 4):
            a.append(0)
        else:
            a.append(1)
        
    elif ((df_train.iloc[n,1] == (df_train.iloc[n-1,1]+1)) and (df_train.iloc[n,1]) == (df_train.iloc[n+1,1]-1)):    #월,금 아니면(1,2,3) 앞행+1값과 뒷행-1값은 모두 일치해야 평일.
        a.append(0)
    else:
        a.append(1)
df_train["공휴일여부"] = a
train_ln = df_train.copy()
train_ln.head(10)

# 주기성 파생변수 생성
time_zero = datetime(1970, 1, 1, 0, 0, 0)
day_to_sec = 24*60*60
year_to_sec = (365.2425)*day_to_sec
frequency_sin_year = []
frequency_cos_year = []
for i in train_x["일자"]:
    time_to_sec = i.to_pydatetime()
    time_interval = (time_to_sec - time_zero).total_seconds()
    frequency_sin_year.append(np.sin((time_interval / year_to_sec) * 2 * np.pi))
    frequency_cos_year.append(np.cos((time_interval / year_to_sec) * 2 * np.pi))

train_x["frequency_sin_year"] = frequency_sin_year
train_x["frequency_cos_year"] = frequency_cos_year
# train_x["frequency_sin_year"].plot()
# train_x["frequency_cos_year"].plot()

# 본사정원수 numeric
# 본사휴가자수 numeric
# 본사출장자수 numeric
# 본사시간외근무명령서승인건수 numeric
# 현본사소속재택근무자수 numeric
# 식사가용인원 numeric (본사정원수-(본사휴가자수+본사출장자수+현본사소속재택근무자수)
# 야근비율 numeric (본사시간외근무명령서승인건수 / 식사가용인원)
# 휴가비율 numeric (본사휴가자수 / 본사정원수)
# 출장비율 numeric (본사출장자수 / 본사정원수)
# 재택비율 numeric (현본사소속재택근무자수 / 본사정원수)

# 식사 가용인원 파생변수 생성 
df_train['식사가용인원'] =df_train['본사정원수']-(df_train['본사휴가자수']+df_train['본사출장자수']+df_train['현본사소속재택근무자수']).astype(int)
df_train['야근비율'] = round(df_train['본사시간외근무명령서승인건수'] / df_train['식사가용인원'],3).astype(float)
df_train['휴가비율'] = round(df_train['본사휴가자수'] / df_train['본사정원수'],3).astype(float)
df_train['출장비율'] = round(df_train['본사출장자수'] / df_train['본사정원수'],3).astype(float)
df_train['재택비율'] = round(df_train['현본사소속재택근무자수'] / df_train['본사정원수'],3).astype(float)
df_train.head()

df_train = read_csv("./2차프로젝트/original/train.csv", parse_dates=["일자"])
df_train['년'] = df_train['일자'].dt.year
df_train['월'] = df_train['일자'].dt.month
df_train['일'] = df_train['일자'].dt.day
df_train['주'] = df_train['일자'].dt.week
df_train['요일'] = df_train['일자'].dt.weekday

# ===== 메뉴 embedding =====
# 일별 점심메뉴를 작은 리스트로 갖고 있는 2중 리스트 (lunch_train) 만들기
lunch_train = []
for day in range(len(df_train)):
    tmp = df_train.loc[day, "중식메뉴"].split(' ')  # 공백으로 문자열 구분
    print(tmp)
    tmp = ' '.join(tmp).split()  # 빈 원소 삭제
    print(tmp)

    search = '('  # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    lunch_train.append(tmp)
lunch_train # 데이터 확인

# lunch train data에 메뉴명별 칼럼 만들기 (밥, 국, 반찬1-3)
menu_len_list = []
bob = []; gook = []; jm = []; side1 = []; side2 = []; kimchi = []; dessert = [];
for i, day_menu in enumerate(lunch_train):
    bob_tmp = day_menu[0]; bob.append(bob_tmp)
    gook_tmp = day_menu[1]; gook.append(gook_tmp)
    jm_tmp = day_menu[2]; jm.append(jm_tmp)
    side1_tmp = day_menu[3]; side1.append(side1_tmp)
    side2_tmp = day_menu[4]; side2.append(side2_tmp)
    if i < 1067:
        kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
        dessert_tmp = day_menu[-2]; dessert.append(dessert_tmp)
    else:
        kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
        dessert_tmp = day_menu[-1]; dessert.append(dessert_tmp)
    menu_len_list.append([len(day_menu),i])
menu_len_list # 데이터 확인
train_ln = df_train.copy()
train_ln['밥'] = bob
train_ln['국'] = gook
train_ln['메인메뉴'] = jm; train_ln['반찬1'] = side1; train_ln['반찬2'] = side2
train_ln['김치'] = kimchi
train_ln['사이드'] = dessert
train_ln.info()
train_ln.head(10)

#========특식 컬럼 추가 (쌀밥이 아닌 비빔밥 등의 특식 메뉴 제공)============
jmt = []; #특식 컬럼 변수명 지정
for i in train_ln['밥']:
    if '쌀밥' not in i:
        jmt.append(1) #’쌀밥’이라는 단어가 ‘밥’ 컬럼에 있는 경우 1을 반환
    else:
        jmt.append(0) #있는 경우 0을 반환
train_ln['특식'] = jmt #따라서 특식에 쌀밥이 아닌 다른 밥메뉴들이 들어갈 경우 1을 반환

#========신메뉴 컬럼 추가 (신메뉴가 수요에 미치는 영향을 파악) (중식)===========
new = []
for i in range(len(train_ln)): #행 길이 만큼 반복
    if 'New' in train_ln.loc[i, '중식메뉴']: #중식메뉴 컬럼에 ‘New’라는 문자열이 있으면
            new.append(1) #1을 반환
    else:
            new.append(0) #아님 0을 반환
train_ln['신메뉴'] = new


## 기상청 외부데이터

In [ ]:
# 인자 -> data_x : iterable, col_names: 찾을 리스트
# 리턴 -> 인덱스 위치를 담은 리스트 
# col_names 를 인자로 받아서 data_x 에 어디 있는지 인덱스를 찾습니다
def findIdx(data_x, col_names):
    return [int(i) for i, j in enumerate(data_x) if j in col_names]

# 기상청 외부데이터 활용
# 점심 11,12,13 저녁 17,18,19
forecast = read_csv("./2차프로젝트/2016_2021_진주_기온강수.csv", encoding="euc-kr")
forecast["강수량"].fillna(0, inplace=True)
forecast.isna().sum()

findIdx(forecast["기온"].isna(), [True])
forecast["기온"][40765:40768]
forecast["기온"][40766] = forecast["기온"][40765:40768].mean()
forecast["기온"][40768:40771]
forecast["기온"][40769] = forecast["기온"][40768:40771].mean()
forecast.isna().sum().sum()
train_x.isna().sum().sum()

# forecast
tmp_list = []
for i in forecast["일시"]:
    if ("11:00" in i) or ("12:00" in i) or ("13:00" in i):
        tmp_list.append(True)
    else:
        tmp_list.append(False)

forecast = forecast[tmp_list]
forecast["일시"] = pd.to_datetime(forecast["일시"])
forecast.set_index("일시", inplace=True)

forecast.isna().sum().sum()

forecast.resample("1D").mean().isna().sum()

# 10:16 사이에 데이터 없음
# 8:15 사이에 데이터 없음
# round(forecast.resample("1D").mean()["기온"], 1)
# forecast.resample("1D").max()["강수량"]
forecast_new = pd.concat([round(forecast.resample("1D").mean()["기온"], 1), forecast.resample("1D").max()["강수량"]], axis=1)
forecast_new.isna().sum()

forecast_new.reset_index("일시", inplace=True)
forecast_new.columns = ["일자", "기온", "강수량"]

# train_x = pd.merge(train_x, forecast_new, left_on="일자")
train_x = pd.merge(train_x, forecast_new, how="left", on="일자")
train_x.isna().sum().sum()
# 기온 및 강수량의 11~13 시 결측치는 전날 기온 및 강수량으로 대체하였다
train_x["기온"][findIdx(train_x["기온"].isna(), [True])[0]] = train_x["기온"][findIdx(train_x["기온"].isna(), [True])[0]-1]
train_x["강수량"][findIdx(train_x["강수량"].isna(), [True])[0]] = 0
train_x.isna().sum().sum()

train_x["강수여부"] = [1 if i>0 else 0 for i in train_x["강수량"]]